A notebook to perform sequence clustering with this data
Source: https://www.ncbi.nlm.nih.gov/sra/SRX26494648%5Baccn%5D


In [4]:
import subprocess

In [ ]:
# may need to alter this when switching to codespace 

# local path
! /Users/milton/anaconda3/bin/fastqc /Users/milton/Documents/University/Dissertation/git/DNA-Data/SRR31112377.fastq -o /Users/milton/Documents/University/Dissertation/git/DNA-Data/

# codespace path
# ! /usr/bin/fastqc /workspaces/DNA-Data/SRR31112377.fastq -o /workspaces/DNA-Data/

null
Started analysis of SRR31112377.fastq
Approx 5% complete for SRR31112377.fastq
Approx 10% complete for SRR31112377.fastq
Approx 15% complete for SRR31112377.fastq
Approx 20% complete for SRR31112377.fastq
Approx 25% complete for SRR31112377.fastq
Approx 30% complete for SRR31112377.fastq
Approx 35% complete for SRR31112377.fastq
Approx 40% complete for SRR31112377.fastq
Approx 45% complete for SRR31112377.fastq
Approx 50% complete for SRR31112377.fastq
Approx 55% complete for SRR31112377.fastq
Approx 60% complete for SRR31112377.fastq
Approx 65% complete for SRR31112377.fastq
Approx 70% complete for SRR31112377.fastq
Approx 75% complete for SRR31112377.fastq
Approx 80% complete for SRR31112377.fastq
Approx 85% complete for SRR31112377.fastq
Approx 90% complete for SRR31112377.fastq
Approx 95% complete for SRR31112377.fastq
Analysis complete for SRR31112377.fastq
